In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import log_loss
import os

/Users/avinash/.local/share/virtualenvs/.kaggle-x1F4j2mL/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
f = {f.split('/')[-1]: pd.read_csv(f, encoding='latin1') for f in os.listdir() if f.endswith('.csv')}
#for df in f: print(df, list(f[df].columns))

teams = f['WTeams.csv']
teams2 = f['WTeamSpellings.csv']
season_cresults = f['WRegularSeasonCompactResults.csv']
season_dresults = f['WRegularSeasonDetailedResults.csv']
tourney_cresults = f['WNCAATourneyCompactResults.csv']
tourney_dresults = f['WNCAATourneyDetailedResults.csv']
slots = f['WNCAATourneySlots.csv']
seeds = f['WNCAATourneySeeds.csv']
seeds = {'_'.join(map(str,[int(k1),k2])):int(v[1:3]) for k1, v, k2 in seeds[['Season', 'Seed', 'TeamID']].values}
seeds = {**seeds, **{k.replace('2021_','2022_'):seeds[k] for k in seeds if '2021_' in k}}
cities = f['Cities.csv']
gcities = f['WGameCities.csv']
seasons = f['WSeasons.csv']
sub = f['WSampleSubmissionStage1.csv']

In [4]:
teams2 = teams2.groupby(by='TeamID', as_index=False)['TeamNameSpelling'].count()
teams2.columns = ['TeamID', 'TeamNameCount']
teams = pd.merge(teams, teams2, how='left', on=['TeamID'])
del teams2

teams.head()

,TeamID,TeamName,TeamNameCount
0,3101,Abilene Chr,3
1,3102,Air Force,2
2,3103,Akron,1
3,3104,Alabama,1
4,3105,Alabama A&M,2


In [5]:
season_cresults['ST'] = 'S'
season_dresults['ST'] = 'S'
tourney_cresults['ST'] = 'T'
tourney_dresults['ST'] = 'T'

In [6]:
# games = pd.concat((season_cresults, tourney_cresults), axis=0, ignore_index=True)
games = pd.concat((season_dresults, tourney_dresults), axis=0, ignore_index=True)
games.reset_index(drop=True, inplace=True)
games['WLoc'] = games['WLoc'].map({'A': 1, 'H': 2, 'N': 0})

games['ID'] = games.apply(lambda r: '_'.join(map(str, [r['Season']]+sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['IDTeams'] = games.apply(lambda r: '_'.join(map(str, sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['Team1'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[0], axis=1)
games['Team2'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[1], axis=1)
games['IDTeam1'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
games['IDTeam2'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)

games['Team1Seed'] = games['IDTeam1'].map(seeds).fillna(0)
games['Team2Seed'] = games['IDTeam2'].map(seeds).fillna(0)

games['ScoreDiff'] = games['WScore'] - games['LScore']
games['Pred'] = games.apply(lambda r: 1. if sorted([r['WTeamID'],r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1)
games['ScoreDiffNorm'] = games.apply(lambda r: r['ScoreDiff'] * -1 if r['Pred'] == 0. else r['ScoreDiff'], axis=1)
games['SeedDiff'] = games['Team1Seed'] - games['Team2Seed']
games['WEFFG'] = games['WFGM'] / games['WFGA']
games['WEFFG3'] = games['WFGM3'] / games['WFGA3']
games['WDARE'] = games['WFGM3'] / games['WFGM']
games['WTOQUETOQUE'] = games['WAst'] / games['WFGM']

games['LEFFG'] = games['LFGM'] / games['LFGA']
games['LEFFG3'] = games['LFGM3'] / games['LFGA3']
games['LDARE'] = games['LFGM3'] / games['LFGM']
games['LTOQUETOQUE'] = games['LAst'] / games['LFGM']
games = games.fillna(-1)
games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,ScoreDiffNorm,SeedDiff,WEFFG,WEFFG3,WDARE,WTOQUETOQUE,LEFFG,LEFFG3,LDARE,LTOQUETOQUE
0,2010,11,3103,63,3237,49,2,0,23,54,...,14,0.0,0.425926,0.555556,0.217391,0.608696,0.370370,0.230769,0.150000,0.550000
1,2010,11,3104,73,3399,68,0,0,26,62,...,5,0.0,0.419355,0.416667,0.192308,0.576923,0.396825,0.190476,0.160000,0.280000
2,2010,11,3110,71,3224,59,1,0,29,62,...,12,0.0,0.467742,0.400000,0.206897,0.620690,0.327586,0.142857,0.105263,0.421053
3,2010,11,3111,63,3267,58,1,0,27,52,...,5,0.0,0.519231,0.363636,0.148148,0.518519,0.243243,0.230769,0.333333,0.833333
4,2010,11,3119,74,3447,70,2,1,30,74,...,4,0.0,0.405405,0.350000,0.233333,0.600000,0.337838,0.529412,0.360000,0.480000


In [7]:
c_score_col = ['NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl',
 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl',
 'LBlk', 'LPF' ,'WEFFG', 'WEFFG3', 'WDARE', 'WTOQUETOQUE', 'LEFFG', 'LEFFG3', 'LDARE', 'LTOQUETOQUE']
c_score_agg = ['sum', 'mean', 'median', 'max', 'min', 'std', 'skew', 'nunique']

gb = games.groupby(by=['IDTeams']).agg({k: c_score_agg for k in c_score_col}).reset_index()
gb.columns = [''.join(c) + '_c_score' for c in gb.columns]

games = games[games['ST']=='T']
games = pd.merge(games, gb, how='left', left_on='IDTeams', right_on='IDTeams_c_score')
col = [c for c in games.columns if c not in ['ID', 'DayNum', 'ST', 'Team1', 'Team2', 'IDTeams', 'IDTeam1', 'IDTeam2', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'NumOT', 'Pred', 'ScoreDiff', 'ScoreDiffNorm', 'WLoc', 'IDTeams_c_score']+c_score_col]

In [8]:
xgb = XGBRegressor()
xgb.fit(games[col].fillna(-1), games['Pred'])
pred = xgb.predict(games[col].fillna(-1)).clip(0.000000001,0.999999999)
print('Log Loss:', log_loss(games['Pred'], pred))

/Users/avinash/.local/share/virtualenvs/.kaggle-x1F4j2mL/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Log Loss: nan


/Users/avinash/.local/share/virtualenvs/.kaggle-x1F4j2mL/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/avinash/.local/share/virtualenvs/.kaggle-x1F4j2mL/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2442: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/Users/avinash/.local/share/virtualenvs/.kaggle-x1F4j2mL/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2442: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


In [9]:
pred = xgb.predict(games[col].fillna(-1)).clip(0.0001,0.9999)
print('Log Loss:', log_loss(games['Pred'], pred.round()))

Log Loss: nan


/Users/avinash/.local/share/virtualenvs/.kaggle-x1F4j2mL/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/avinash/.local/share/virtualenvs/.kaggle-x1F4j2mL/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2442: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/Users/avinash/.local/share/virtualenvs/.kaggle-x1F4j2mL/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2442: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


In [10]:
sub = f['WSampleSubmissionStage1.csv']
sub['WLoc'] = 2 #sub['WLoc'].map({'A': 1, 'H': 2, 'N': 0})
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['Season'].astype(int)
sub['Team1'] = sub['ID'].map(lambda x: x.split('_')[1])
sub['Team2'] = sub['ID'].map(lambda x: x.split('_')[2])
sub['IDTeams'] = sub.apply(lambda r: '_'.join(map(str, [r['Team1'], r['Team2']])), axis=1)
sub['IDTeam1'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
sub['IDTeam2'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
sub['Team1Seed'] = sub['IDTeam1'].map(seeds).fillna(0)
sub['Team2Seed'] = sub['IDTeam2'].map(seeds).fillna(0)
sub['SeedDiff'] = sub['Team1Seed'] - sub['Team2Seed'] 

sub = sub.fillna(-1)
sub = pd.merge(sub, gb, how='left', left_on='IDTeams', right_on='IDTeams_c_score')

In [11]:
sub['Pred'] = xgb.predict(sub[col].fillna(-1)).clip(0.0001,0.9999).round()
sub[['ID', 'Pred']].to_csv('xgbwsubmission.csv', index=False)

/Users/avinash/.local/share/virtualenvs/.kaggle-x1F4j2mL/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
